## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Longyearbyen,SJ,78.2186,15.6401,30.04,64,75,3.44,broken clouds
1,1,Beringovskiy,RU,63.0500,179.3167,35.98,96,100,5.75,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,20.70,69,99,0.89,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,58.32,69,100,8.16,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.46,77,52,18.79,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 65


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Mar Del Plata,AR,-38.0023,-57.5575,61.95,38,0,12.66,clear sky
46,46,Gillette,US,44.2911,-105.5022,61.11,36,94,5.01,overcast clouds
56,56,Semey,KZ,50.4111,80.2275,63.88,60,100,0.38,overcast clouds
57,57,Carnarvon,AU,-24.8667,113.6333,64.29,50,0,17.13,clear sky
63,63,Miles City,US,46.4083,-105.8406,60.69,24,2,17.05,clear sky
70,70,Hobart,AU,-42.8794,147.3294,60.26,76,40,2.30,scattered clouds
71,71,Airai,TL,-8.9266,125.4092,61.54,97,73,0.43,broken clouds
72,72,Vila Franca Do Campo,PT,37.7167,-25.4333,64.20,94,100,24.90,moderate rain
85,85,Luderitz,NaN,-26.6481,15.1594,62.69,48,27,14.72,scattered clouds
99,99,Berlin,DE,52.5244,13.4105,64.42,62,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.replace("", np.nan, inplace=True)
preferred_cities_df.isnull().sum()

C:\Users\laure\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Unnamed: 0             0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset=["Country"], inplace=True)
preferred_cities_df.isnull().sum()

C:\Users\laure\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Mar Del Plata,AR,61.95,clear sky,-38.0023,-57.5575,
46,Gillette,US,61.11,overcast clouds,44.2911,-105.5022,
56,Semey,KZ,63.88,overcast clouds,50.4111,80.2275,
57,Carnarvon,AU,64.29,clear sky,-24.8667,113.6333,
63,Miles City,US,60.69,clear sky,46.4083,-105.8406,
70,Hobart,AU,60.26,scattered clouds,-42.8794,147.3294,
71,Airai,TL,61.54,broken clouds,-8.9266,125.4092,
72,Vila Franca Do Campo,PT,64.20,moderate rain,37.7167,-25.4333,
99,Berlin,DE,64.42,clear sky,52.5244,13.4105,
115,Cape Town,ZA,60.91,clear sky,-33.9258,18.4232,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))